<a href="https://colab.research.google.com/github/cddolanc/cct-start/blob/main/House_Price_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

In [2]:
dataset = pd.read_csv('kc_house_data.csv')

In [3]:
dataset.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
X = dataset.iloc[:, 3:].values
X = X[:, np.r_[0:13,14:18]] # get rid of row 13 continue X from 14 to 18
y = dataset.iloc[:, 2].values


In [5]:
X # columns

array([[ 3.00000e+00,  1.00000e+00,  1.18000e+03, ..., -1.22257e+02,
         1.34000e+03,  5.65000e+03],
       [ 3.00000e+00,  2.25000e+00,  2.57000e+03, ..., -1.22319e+02,
         1.69000e+03,  7.63900e+03],
       [ 2.00000e+00,  1.00000e+00,  7.70000e+02, ..., -1.22233e+02,
         2.72000e+03,  8.06200e+03],
       ...,
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ..., -1.22299e+02,
         1.02000e+03,  2.00700e+03],
       [ 3.00000e+00,  2.50000e+00,  1.60000e+03, ..., -1.22069e+02,
         1.41000e+03,  1.28700e+03],
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ..., -1.22299e+02,
         1.02000e+03,  1.35700e+03]])

In [6]:
y # price

array([221900., 538000., 180000., ..., 402101., 400000., 325000.])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) #

X y train  test size is 0.2 or 20%

In [8]:
xscaler = MinMaxScaler(feature_range = (0,1))
X_train = xscaler.fit_transform(X_train) # xscaler set here
X_test = xscaler.transform(X_test)

Scale your data between 0 and 1


In [9]:
yscaler = MinMaxScaler(feature_range = (0,1))
y_train = yscaler.fit_transform(y_train.reshape(-1,1)) # yscaler set here
y_test = yscaler.transform(y_test.reshape(-1,1))

In [10]:
model = Sequential() # create a dense neural network
model.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu', input_dim = 17)) # layer 1, 17 features
model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu')) # layer 2
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu')) # layer 3
model.compile(optimizer = Adam(lr = 0.001), loss = 'mse', metrics = ['mean_absolute_error']) # optimizer Adam used

In [11]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
541/541 [==============================] - 2s 3ms/step - loss: 0.0017 - mean_absolute_error: 0.0259 - val_loss: 6.1505e-04 - val_mean_absolute_error: 0.0161
Epoch 2/100
541/541 [==============================] - 1s 2ms/step - loss: 6.4487e-04 - mean_absolute_error: 0.0157 - val_loss: 5.2875e-04 - val_mean_absolute_error: 0.0145
Epoch 3/100
541/541 [==============================] - 1s 2ms/step - loss: 6.0613e-04 - mean_absolute_error: 0.0152 - val_loss: 5.0388e-04 - val_mean_absolute_error: 0.0136
Epoch 4/100
541/541 [==============================] - 1s 2ms/step - loss: 5.6836e-04 - mean_absolute_error: 0.0146 - val_loss: 4.9963e-04 - val_mean_absolute_error: 0.0141
Epoch 5/100
541/541 [==============================] - 1s 2ms/step - loss: 5.6055e-04 - mean_absolute_error: 0.0146 - val_loss: 4.8122e-04 - val_mean_absolute_error: 0.0137
Epoch 6/100
541/541 [==============================] - 1s 2ms/step - loss: 5.4102e-04 - mean_absolute_error: 0.0144 - val_loss: 4.7531e-04 

In [12]:
y_test = yscaler.inverse_transform(y_test)
prediction = yscaler.inverse_transform(model.predict(X_test))

In [13]:
error = abs(prediction - y_test)/y_test
print(np.mean(error))

0.16907733238114758
